# Persistence

Many AI applications need memory to share context across multiple interactions. In LangGraph, memory is provided for any [StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html) through [Checkpointers](https://langchain-ai.github.io/langgraphjs/reference/interfaces/index.Checkpoint.html).

When creating any LangGraph workflow, you can set them up to persist their state by doing using the following:

1. A [Checkpointer](https://langchain-ai.github.io/langgraphjs/reference/classes/index.BaseCheckpointSaver.html), such as the [MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html)
2. Call `compile(checkpointer=myCheckpointer)` when compiling the graph.

<!-- Example:
```javascript
from langgraph.graph import StateGraph
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

builder = StateGraph(....)
# ... define the graph
memory = AsyncSqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)
...
``` -->

This works for [StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html) and all its subclasses, such as [MessageGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MessageGraph.html).

Below is an example.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>create_react_agent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>

## Setup

This guide will use Anthropic's Claude model. We will optionally set our API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.


In [1]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Persistence: LangGraphJS");


## Define the state

The state is the interface for all of the nodes in our graph.


In [2]:
import { BaseMessage } from "@langchain/core/messages";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  next: string;
}

// This defines the agent state
const State: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};


## Set up the tools

We will first define the tools we want to use. For this simple example, we will use create a placeholder search engine.
However, it is really easy to create your own tools - see documentation [here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do that.


In [3]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return ["Cold, with a low of -78 ℃"];
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];


[ "Cold, with a low of -78 ℃" ]

We can now wrap these tools in a simple [ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html). This object will actually run the tools (functions) whenever they are invoked by our LLM.


In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);


## Set up the model

Now we will load the [chat model]().
https://js.langchain.com/v0.2/docs/concepts/#chat-models

1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with [tool calling](https://js.langchain.com/v0.2/docs/how_to/tool_calling/#passing-tools-to-llms), meaning it can return function arguments in its response.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        These model requirements are not general requirements for using LangGraph - they are just requirements for this one example.
    </p>
</div>


In [5]:
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" });


After we've done this, we should make sure the model knows that it has these tools available to call. We can do this by calling [bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools).


In [6]:
const boundModel = model.bindTools(tools);
boundModel.kwargs;


{
  tools: [
    {
      name: "search",
      description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
      input_schema: {
        type: "object",
        properties: { query: [Object] },
        required: [ "query" ],
        additionalProperties: false,
        "$schema": "http://json-schema.org/draft-07/schema#"
      }
    }
  ]
}

## Define the graph

We can now put it all together. We will run it first without a checkpointer:


In [7]:
import { StateGraph, END, START } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const routeMessage = (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage.tool_calls.length) {
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "tools";
};

const callModel = async (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages);
  return { messages: [response] };
};

const workflow = new StateGraph({
  channels: State,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("tools", toolNode);

// Set the entrypoint as `agent`
workflow.addEdge(START, "agent");
workflow.addConditionalEdges("agent", routeMessage);
workflow.addEdge("tools", "agent");

const graph = workflow.compile();
("");


""

In [8]:
let inputs = { messages: [["user", "Hi I'm Yu, niced to meet you."]] };

for await (const { messages } of await graph.stream(inputs, {
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


[ "user", "Hi I'm Yu, niced to meet you." ]
-----

It's nice to meet you too, Yu! I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. I'm here to assist you with any questions or tasks you may have. Please let me know how I can be of help!
-----



In [9]:
inputs = { messages: [["user", "Remember my name?"]] };

for await (const { messages } of await graph.stream(inputs, {
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


[ "user", "Remember my name?" ]
-----

I'm afraid I don't actually have a way to remember your name, since this is a new conversation. As an AI assistant without persistent memory, I don't have a way to keep track of details about previous interactions. Could you please provide your name again so I can refer to you properly?
-----



## Add Memory

Let's try it again with a checkpointer. We will use the SqliteSaver, which will "save" checkpoints in-memory.


In [10]:
import { MemorySaver } from "@langchain/langgraph";
import { load } from "@langchain/core/load";

// Here we only save in-memory
const memory = new MemorySaver({
  parse: load,
  stringify: JSON.stringify,
});
const persistentGraph = workflow.compile({ checkpointer: memory });


In [11]:
let config = { configurable: { thread_id: "conversation-num-1" } };
inputs = { messages: [["user", "Hi I'm Jo, niced to meet you."]] };
for await (const { messages } of await persistentGraph.stream(inputs, {
  ...config,
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


[ "user", "Hi I'm Jo, niced to meet you." ]
-----

It's nice to meet you too, Jo! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist you with.
-----



In [13]:
inputs = { messages: [["user", "Remember my name?"]] };
for await (const { messages } of await persistentGraph.stream(inputs, {
  ...config,
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


[ "user", "Remember my name?" ]
-----

Yes, I remember your name is Jo. It's nice to meet you Jo!
-----



## New Conversational Thread

If we want to start a new conversation, we can pass in a different **`thread_id`**. Poof! All the memories are gone (just kidding, they'll always live in that other thread)!


In [14]:
config = { configurable: { thread_id: "conversation-2" } };


{ configurable: { thread_id: "conversation-2" } }

In [15]:
inputs = { messages: [["user", "you forgot?"]] };
for await (const { messages } of await persistentGraph.stream(inputs, {
  ...config,
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


[ "user", "you forgot?" ]
-----

I'm afraid I don't have enough context to understand what you mean by "you forgot?". Could you please provide more details about what you are referring to? I'd be happy to try and assist you once I have a better understanding of your question or concern.
-----

